In [1]:
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random

from shapely.geometry import Point


In [3]:

def create_stratified_samples(raster_path, aoi_path, out_path, num_samples=100):
    # Load the AOI shapefile
    aoi = gpd.read_file(aoi_path)
    
    # Open the flood binary raster
    with rasterio.open(raster_path) as src:
        flood_data = src.read(1)
        transform = src.transform
    
    # Mask the raster with AOI
    mask = geometry_mask([aoi.geometry.unary_union], transform=transform, invert=True, out_shape=flood_data.shape)
    masked_flood_data = flood_data[mask]
    
    # Extract coordinates of masked data
    indices = np.column_stack(np.where(mask))
    coordinates = [Point(rasterio.transform.xy(transform, i, j)) for i, j in indices]
    flood_values = masked_flood_data[mask]
    
    # Create DataFrame of points and flood values
    df = pd.DataFrame({
        'geometry': coordinates,
        'flood_value': flood_values
    })
    
    # Convert DataFrame to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    
    # Stratified sampling
    samples_per_class = num_samples // 2  # Assuming equal samples for both classes
    flooded_samples = gdf[gdf['flood_value'] == 1].sample(n=samples_per_class, random_state=42)
    non_flooded_samples = gdf[gdf['flood_value'] == 0].sample(n=samples_per_class, random_state=42)
    
    # Combine samples
    samples_gdf = pd.concat([flooded_samples, non_flooded_samples])
    
    # Save to shapefile
    samples_gdf.to_file(f'{out_path}/{name}_samples.shp')
    print(samples_gdf.head())
    print("Stratified sample points saved to stratified_samples.shp")


In [4]:

# Paths to the raster and AOI shapefile
raster_path = "e:/FMSE_local/data/rasters/Shikarpur_flood_mapped_layer.tif"
aoi_path = "e:/FMSE_local/data/shp/Shikarpur.shp"
out_path = 'e:/FMSE_local/data/shp/sample_points/'
# Create stratified samples
name = "Shikarpur"
create_stratified_samples(raster_path, aoi_path, out_path)
